In [55]:



from google.colab import drive
drive.mount('/content/drive')

import os
from google.colab import drive

# Mount google drive
DRIVE_MOUNT='/content/gdrive'
drive.mount(DRIVE_MOUNT)


# create folder to write data to
CIS_565_FOLDER=os.path.join(DRIVE_MOUNT, 'My Drive', 'CIS_565_2019')
FINAL_PROJECT_FOLDER=os.path.join(CIS_565_FOLDER, 'Final_Project')
os.makedirs(FINAL_PROJECT_FOLDER, exist_ok=True)

# bootstrap environment into place
from google.colab import auth
auth.authenticate_user()

from googleapiclient.discovery import build
drive_service = build('drive', 'v3')

import io
import os
from googleapiclient.http import MediaIoBaseDownload

def download_file(fn, file_id):
    request = drive_service.files().get_media(fileId=file_id)
    downloaded = io.BytesIO()
    downloader = MediaIoBaseDownload(downloaded, request)
    done = False
    while done is False:
        # _ is a placeholder for a progress object that we ignore.
        # (Our file is small, so we skip reporting progress.)
        _, done = downloader.next_chunk()
    
    downloaded.seek(0)

    folder = fn.split('/')
    if len(folder) > 1:
        os.makedirs(folder[0], exist_ok=True)

    with open(fn, 'wb') as f:
        f.write(downloaded.read())

id_to_fn = {}

# download all files into the vm
for fid, fn in id_to_fn.items():
    download_file(fn, fid)


Mounted at /content/drive
Mounted at /content/gdrive


In [0]:
!wget -nc -P '/content/gdrive/My Drive/CIS_565_2019/Final_Project/' "https://shapenet.cs.stanford.edu/media/modelnet40_ply_hdf5_2048.zip"
!unzip -n '/content/gdrive/My Drive/CIS_565_2019/Final_Project/modelnet40_ply_hdf5_2048.zip' -d "/content/gdrive/My Drive/CIS_565_2019/Final_Project/data/"

File ‘/content/gdrive/My Drive/CIS_565_2019/Final_Project/modelnet40_ply_hdf5_2048.zip’ already there; not retrieving.

Archive:  /content/gdrive/My Drive/CIS_565_2019/Final_Project/modelnet40_ply_hdf5_2048.zip


In [0]:
!rm -r -f ./graph
import os
import h5py
import numpy as np
import scipy
from scipy import sparse
import scipy.sparse.linalg as sp
import numpy as np
from scipy.spatial import cKDTree
import pickle

class Parameters():
  def __init__(self):
    self.neighborNumber = 40
    self.outputClassN = 10
    self.pointNumber = 1024
    self.gcn_1_filter_n = 1000
    self.gcn_2_filter_n = 1000
    self.gcn_3_filter_n = 1000
    self.fc_1_n = 600
    self.chebyshev_1_Order = 4
    self.chebyshev_2_Order = 3
    self.keep_prob_1 = 0.9 #0.9 original
    self.keep_prob_2 = 0.55
    self.batchSize = 28
    self.testBatchSize = 1
    self.max_epoch = 260
    self.learningRate = 12e-4
    self.dataset = 'ModelNet40'
    self.weighting_scheme = 'weighted'
    self.modelDir = '/raid60/yingxue.zhang2/ICASSP_code/global_pooling/model/'
    self.logDir = '/raid60/yingxue.zhang2/ICASSP_code/global_pooling/log/'
    self.fileName = '0112_1024_40_cheby_4_3_modelnet40_max_var_first_second_layer'
    self.weight_scaler = 40#50

def adjacency(dist, idx):
    """Return the adjacency matrix of a kNN graph."""
    M, k = dist.shape
    assert M, k == idx.shape
    assert dist.min() >= 0
    # Weights.
    sigma2 = np.mean(dist[:, -1]) ** 2
    #print sigma2
    dist = np.exp(- dist ** 2 / sigma2)

    # Weight matrix.
    I = np.arange(0, M).repeat(k)
    J = idx.reshape(M * k)
    V = dist.reshape(M * k)
    W = scipy.sparse.coo_matrix((V, (I, J)), shape=(M, M))
    # No self-connections.
    W.setdiag(0)

    # Non-directed graph.
    bigger = W.T > W
    W = W - W.multiply(bigger) + W.T.multiply(bigger)
    return W

def normalize_adj(adj):
    adj = sparse.coo_matrix(adj)
    rowsum = np.array(adj.sum(1))
    d_inv_sqrt = np.power(rowsum, -0.5).flatten()
    d_inv_sqrt[np.isinf(d_inv_sqrt)] = 0.
    d_mat_inv_sqrt = sparse.diags(d_inv_sqrt)
    return adj.dot(d_mat_inv_sqrt).transpose().dot(d_mat_inv_sqrt).tocoo()

def normalized_laplacian(adj):
    adj_normalized = normalize_adj(adj)
    norm_laplacian = sparse.eye(adj.shape[0]) - adj_normalized
    return norm_laplacian

def scaled_laplacian(adj):
    adj_normalized = normalize_adj(adj)
    laplacian = sparse.eye(adj.shape[0]) - adj_normalized
    largest_eigval, _ = sp.eigsh(laplacian, 1, which='LM')
    scaled_laplacian = (2. / largest_eigval[0]) * laplacian - scipy.sparse.eye(adj.shape[0])
    return scaled_laplacian


def getDataFiles(list_filename):
    return [line.rstrip() for line in open(list_filename)]

def load_h5(h5_filename):
    f = h5py.File(os.path.join(FINAL_PROJECT_FOLDER,h5_filename))
    data = f['data'][:]
    label = f['label'][:]
    return (data, label)

def loadDataFile(filename):
    return load_h5(filename)
    
    
def farthestSampling(file_names, NUM_POINT):
    file_indexs = np.arange(0, len(file_names))
    inputData = dict()
    inputLabel = dict()
    for index in range (len(file_indexs)):
        current_data, current_label = loadDataFile(file_names[file_indexs[index]])
        current_data = current_data[:,0:NUM_POINT,:]
        current_label = np.squeeze(current_label) 
        current_label= np.int_(current_label)
        inputData.update({index : current_data})
        inputLabel.update({index : current_label})
    return inputData, inputLabel

def load_data(NUM_POINT):
    
    TRAIN_FILES = getDataFiles(os.path.join(FINAL_PROJECT_FOLDER, 'data/modelnet40_ply_hdf5_2048/train_files.txt'))
    TEST_FILES = getDataFiles(os.path.join(FINAL_PROJECT_FOLDER, 'data/modelnet40_ply_hdf5_2048/test_files.txt'))
    
    
    inputTrainFarthest, inputTrainLabel = farthestSampling(TRAIN_FILES, NUM_POINT)
    inputTestFathest, inputTestLabel = farthestSampling(TEST_FILES, NUM_POINT)
    return inputTrainFarthest, inputTrainLabel, inputTestFathest, inputTestLabel



para = Parameters()
pointNumber = para.pointNumber
neighborNumber = para.neighborNumber

#generate graph structure and store in the system
def prepareGraph(inputData, neighborNumber, pointNumber, dataType):
    scaledLaplacianDict = dict()
    baseDir = FINAL_PROJECT_FOLDER
    fileDir =  baseDir + '/graph/' + dataType+'_pn_'+str(pointNumber)+'_nn_'+str(neighborNumber)
        
    if (os.path.isdir(fileDir)):
        print ("calculating the graph data")
        # os.makedirs(fileDir)
        for batchIndex in range(len(inputData)):
            batchInput = inputData[batchIndex]
            print("len: ", len(batchInput))
            batchFlattenLaplacian = []
            for i in range(10):
                print (i)
                pcCoordinates = batchInput[i]
                tree = cKDTree(pcCoordinates)
                dd, ii = tree.query(pcCoordinates, k = neighborNumber)
                A = adjacency(dd, ii)
                scaledLaplacian = scaled_laplacian(A)
                scaledLaplacian = scaledLaplacian.toarray()
                print("Lap: ", scaledLaplacian.shape)
                # flattenLaplacian = scaledLaplacian.tolil().reshape((1, pointNumber*pointNumber))
                batchFlattenLaplacian.append(scaledLaplacian)
                # if i ==0:
                #     batchFlattenLaplacian = scaledLaplacian
                # else:
                #     batchFlattenLaplacian = np.stack((batchFlattenLaplacian, scaledLaplacian), axis=-1)
            batchFlattenLaplacian = np.stack(batchFlattenLaplacian, axis=0)
            print("batch lap: ", batchFlattenLaplacian.shape)
            scaledLaplacianDict.update({batchIndex: batchFlattenLaplacian})
            with open(fileDir+'/batchGraph_'+str(batchIndex), 'wb') as handle:
                pickle.dump(batchFlattenLaplacian, handle)
            print ("Saving the graph data batch"+str(batchIndex))
        
    else:
        print("Loading the graph data from "+dataType+'Data')
        scaledLaplacianDict = loadGraph(inputData, neighborNumber, pointNumber, fileDir)
    return scaledLaplacianDict

def get_laplacian_for_batch(batchInput):
  batchFlattenLaplacian = []
  for i in range(len(batchInput)):
      pcCoordinates = batchInput[i]
      tree = cKDTree(pcCoordinates)
      dd, ii = tree.query(pcCoordinates, k = neighborNumber)
      A = adjacency(dd, ii)
      scaledLaplacian = scaled_laplacian(A)
      scaledLaplacian = scaledLaplacian.toarray()
      batchFlattenLaplacian.append(scaledLaplacian)
  batchFlattenLaplacian = np.stack(batchFlattenLaplacian, axis=0)

  return batchFlattenLaplacian

def loadGraph(inputData, neighborNumber, pointNumber, fileDir):
    scaledLaplacianDict = dict()
    for batchIndex in range(len(inputData)):
        batchDataDir = fileDir+'/batchGraph_'+str(batchIndex)
        with open(batchDataDir, 'rb') as handle:
            batchGraph = pickle.load(handle)
        scaledLaplacianDict.update({batchIndex: batchGraph })
        print ("Finish loading batch_"+str(batchIndex))
    return scaledLaplacianDict

                        
def prepareData(inputTrain, inputTest, neighborNumber, pointNumber):
    scaledLaplacianTrain = prepareGraph(inputTrain, neighborNumber, pointNumber, 'train',)
    scaledLaplacianTest = prepareGraph(inputTest, neighborNumber, pointNumber, 'test')
    return scaledLaplacianTrain, scaledLaplacianTest

In [0]:
inputTrain, trainLabel, inputTest, testLabel = load_data(pointNumber)
# scaledLaplacianTrain, scaledLaplacianTest = prepareData(inputTrain, inputTest, neighborNumber, pointNumber)

In [0]:
import torch

def convert_data_to_tensor(input, is_graph = False):
  all_inputs = None
  for i in range(len(input)):
    if is_graph:
      ip = torch.sparse.FloatTensor(input[i])
      ip = torch.reshape(ip, (-1, 1024, 1024))
    else:
      ip = torch.Tensor(input[i])
    if all_inputs is None:
      all_inputs = ip
    else:
      all_inputs = torch.cat((all_inputs, ip))
  return all_inputs

inputTrainTensor = convert_data_to_tensor(inputTrain)
trainLabelTensor = convert_data_to_tensor(trainLabel)

mask = (trainLabelTensor == 1) | (trainLabelTensor == 2) | (trainLabelTensor == 8) | (trainLabelTensor == 12) | (trainLabelTensor == 14) | (trainLabelTensor == 22) | (trainLabelTensor == 23) | (trainLabelTensor == 30)| (trainLabelTensor == 33) | (trainLabelTensor == 35)

# first10mask = (trainLabelTensor < 10)

number_of_samples = 2097

inputTrainTensor = inputTrainTensor[mask]
inputTrainTensor = inputTrainTensor[0:number_of_samples]
trainLabelTensor = trainLabelTensor[mask]
trainLabelTensor = trainLabelTensor[0:number_of_samples]
print(len(mask), len(inputTrainTensor), len(trainLabelTensor), inputTrainTensor.shape)
print(trainLabelTensor)

label_map = {1:0, 2:1, 8:2, 12:3, 14:4, 22:5, 23:6, 30:7, 33:8, 35:9}
trainLabelTensorNew = torch.tensor([label_map[i.item()] for i in trainLabelTensor])
trainLabelTensor = trainLabelTensorNew

inputTestTensor = convert_data_to_tensor(inputTest)
testLabelTensor = convert_data_to_tensor(testLabel)

mask = (testLabelTensor == 1) | (testLabelTensor == 2) | (testLabelTensor == 8) | (testLabelTensor == 12) | (testLabelTensor == 14) | (testLabelTensor == 22) | (testLabelTensor == 23) | (testLabelTensor == 30)| (testLabelTensor == 33) | (testLabelTensor == 35)


number_of_samples = 100

inputTestTensor = inputTestTensor[mask]
inputTestTensor = inputTestTensor[0:number_of_samples]
testLabelTensor = testLabelTensor[mask]
testLabelTensor = testLabelTensor[0:number_of_samples]
print(len(first10mask), len(inputTestTensor), len(testLabelTensor), testLabelTensor.shape)

testLabelTensorNew = torch.tensor([label_map[i.item()] for i in testLabelTensor])
testLabelTensor = testLabelTensorNew
print(testLabelTensor)

9840 2097 2097 torch.Size([2097, 1024, 3])
tensor([30., 30., 22.,  ..., 30.,  8.,  8.])
2468 100 100 torch.Size([100])
tensor([1, 2, 6, 9, 9, 8, 9, 7, 2, 0, 6, 9, 5, 1, 3, 8, 7, 6, 7, 4, 4, 8, 2, 4,
        5, 8, 6, 9, 9, 1, 5, 5, 1, 3, 8, 1, 9, 1, 7, 3, 7, 3, 9, 3, 9, 9, 2, 6,
        2, 9, 9, 9, 5, 3, 1, 5, 5, 4, 4, 1, 2, 0, 3, 3, 6, 7, 4, 9, 9, 9, 2, 9,
        7, 0, 7, 5, 8, 0, 2, 4, 5, 4, 6, 4, 9, 9, 4, 8, 2, 5, 5, 0, 4, 1, 6, 9,
        8, 1, 1, 0])


In [0]:
class ModelNetDataset(torch.utils.data.Dataset):
  def __init__(self, inputData, inputLabel):
    self.data = inputData
    self.labels = inputLabel

  def __len__(self):
    return self.data.shape[0]

  def __getitem__(self, index):
    sample = {'data': self.data[index], 'label': self.labels[index]}
    return sample

modelnetdata = ModelNetDataset(inputTrainTensor, trainLabelTensor)
modelnetdataTest = ModelNetDataset(inputTestTensor, testLabelTensor)


In [0]:
from torch.utils.data import DataLoader

set_batch_size = 32
training_data_batches = DataLoader(modelnetdata, batch_size = set_batch_size, shuffle = True)
test_data_batches = DataLoader(modelnetdataTest, batch_size = 1, shuffle = False)
print(training_data_batches)

In [0]:
device =  torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [0]:
import torch
from torch import nn

class globalPooling(torch.nn.Module):

  def __init__(self, featureNumber):
    super().__init__()
    self.featureNumber = featureNumber

  def forward(self, gcnOutput):
    var = torch.var(gcnOutput, dim = 1) ** 2
    max_f = torch.max(gcnOutput, dim = 1)
    max_f = max_f.values
    poolingOutput = torch.cat((max_f, var), dim = 1)
    return poolingOutput




class gcnLayer(torch.nn.Module):

  def __init__(self,pointNumber, inputFeatureN, outputFeatureN, chebyshev_order):
    super().__init__()
    self.pointNumber      = pointNumber
    self.inputFeatureN    = inputFeatureN
    self.outputFeatureN   = outputFeatureN
    self.chebyshev_order  = chebyshev_order
    self.relu             = nn.ReLU()

    self.biasWeight = nn.Parameter(torch.normal(torch.zeros(self.outputFeatureN), 0.05 * torch.ones(self.outputFeatureN)))
    self.chebyshevCoeff = dict()
    for i in range(chebyshev_order):
        self.chebyshevCoeff['w_' + str(i)] = nn.Parameter(torch.normal(torch.zeros(self.inputFeatureN,self.outputFeatureN), 0.05 * torch.ones(self.inputFeatureN,self.outputFeatureN)))
        self.chebyshevCoeff['w_' + str(i)] = self.chebyshevCoeff['w_' + str(i)].to(device)

  def forward(self,inputPC, scaledLaplacian):
    chebyPoly = []
    cheby_K_Minus_1 = torch.matmul(scaledLaplacian, inputPC)
    cheby_K_Minus_2 = inputPC
    chebyPoly.append(cheby_K_Minus_2)
    chebyPoly.append(cheby_K_Minus_1)
    for i in range(2, self.chebyshev_order):
        chebyK = 2 * (torch.matmul(scaledLaplacian, cheby_K_Minus_1)) - cheby_K_Minus_2
        chebyPoly.append(chebyK)
        cheby_K_Minus_2 = cheby_K_Minus_1
        cheby_K_Minus_1 = chebyK

    gcnOutput = self.biasWeight
    chebyOutput = []
    for i in range(self.chebyshev_order):
        weights = self.chebyshevCoeff['w_' + str(i)]
        chebyPolyReshape = torch.reshape(chebyPoly[i], (-1, self.inputFeatureN))
        output = torch.matmul(chebyPolyReshape, weights)
        output = torch.reshape(output, (-1, self.pointNumber, self.outputFeatureN))
        gcnOutput = gcnOutput + output
        # chebyOutput.append(output)

    gcnOutput = self.relu(gcnOutput)
    
    return gcnOutput


class pointGCN(torch.nn.Module):

  def __init__(self, para):
    super().__init__()
    self.globalFeatureN                   = (para.gcn_1_filter_n + para.gcn_2_filter_n)*2 
    self.gcn1_layer                       = gcnLayer(para.pointNumber, 3, para.gcn_1_filter_n, para.chebyshev_1_Order)
    self.gcn1_dropout_layer               = nn.Dropout(p = 1 - para.keep_prob_1)
    self.gcn1_global_pooling_layer        = globalPooling(para.gcn_1_filter_n)
    self.gcn2_layer                       = gcnLayer(para.pointNumber, para.gcn_1_filter_n, para.gcn_2_filter_n, para.chebyshev_2_Order)
    self.gcn2_dropout_layer               = nn.Dropout(p = 1 - para.keep_prob_1)
    self.gcn2_global_pooling_layer        = globalPooling(para.gcn_2_filter_n)
    self.global_features_dropout_layer    = nn.Dropout(p = 1 - para.keep_prob_2)
    globalFeatureN = (para.gcn_1_filter_n + para.gcn_2_filter_n)*2 
    self.fc1                              = nn.Linear(globalFeatureN, para.fc_1_n)
    self.relu                             = nn.ReLU()
    self.fc1_droput                       = nn.Dropout(p = 1 - para.keep_prob_2)
    self.fc2                              = nn.Linear(para.fc_1_n, para.outputClassN)



  def forward(self, inputPC, scaledLaplacian):
    
    gcn_1         = self.gcn1_layer(inputPC, scaledLaplacian)
    gcn_1_output  = self.gcn1_dropout_layer(gcn_1)
    
    gcn_1_pooling = self.gcn1_global_pooling_layer(gcn_1_output)
    
    gcn_2         = self.gcn2_layer(gcn_1_output, scaledLaplacian)
    gcn_2_output  = self.gcn2_dropout_layer(gcn_2)
    gcn_2_pooling = self.gcn2_global_pooling_layer(gcn_2_output)

    globalFeatures = torch.cat((gcn_1_pooling, gcn_2_pooling), dim=1)
    globalFeatures = self.global_features_dropout_layer(globalFeatures)

    # fully connected layer 1
    fc_layer_1 = self.fc1(globalFeatures)
    fc_layer_1 = self.relu(fc_layer_1)
    fc_layer_1 = self.fc1_droput(fc_layer_1)

    # fully connected layer 2
    fc_layer_2 = self.fc2(fc_layer_1)

    return fc_layer_2


In [0]:
import time

def train(gcn_model, training_data_batches, ce_loss, optimizer, num_epochs):
  for epoch in range(num_epochs):
    epoch_loss = 0
    start = time.time()
    for i, sample in enumerate(training_data_batches):
      batchInput = sample['data']
      batchLabel = sample['label']
      batchLap = torch.Tensor(get_laplacian_for_batch(sample['data']))

      batchInput = batchInput.cuda()
      batchLabel = batchLabel.cuda()
      batchLap = batchLap.cuda()
      
      optimizer.zero_grad()

      batchOutput = gcn_model(batchInput, batchLap)

      loss = ce_loss(batchOutput, batchLabel.long())
      loss.backward()
      optimizer.step()

      epoch_loss += loss.item()

      print("Epoch ", epoch, "Iteration ", i, "Loss: ", loss.item())
    end = time.time()
    time_per_epoch = end - start
    epoch_loss /= len(training_data_batches)
    print("*********************************************************************")
    print("EPOCH ", epoch, " ==> Loss: ", epoch_loss, " Time taken ==> ", time_per_epoch)
    print("*********************************************************************")
  return gcn_model          

In [54]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

num_epochs = 50

gcn_model = pointGCN(para).to(device)
ce_loss = nn.CrossEntropyLoss()
optimizer = optim.Adam(gcn_model.parameters(), lr = 2e-3)

trained_gcn_model = train(gcn_model, training_data_batches, ce_loss, optimizer, num_epochs) 

Epoch  0 Iteration  0 Loss:  2.2876498699188232
Epoch  0 Iteration  1 Loss:  2.2292990684509277
Epoch  0 Iteration  2 Loss:  2.394721508026123
Epoch  0 Iteration  3 Loss:  2.2200677394866943
Epoch  0 Iteration  4 Loss:  2.1301491260528564
Epoch  0 Iteration  5 Loss:  2.1640663146972656
Epoch  0 Iteration  6 Loss:  2.083338975906372
Epoch  0 Iteration  7 Loss:  2.0299620628356934
Epoch  0 Iteration  8 Loss:  2.056279182434082
Epoch  0 Iteration  9 Loss:  1.9493346214294434
Epoch  0 Iteration  10 Loss:  1.9061641693115234
Epoch  0 Iteration  11 Loss:  2.25230073928833
Epoch  0 Iteration  12 Loss:  2.155426025390625
Epoch  0 Iteration  13 Loss:  2.2294809818267822
Epoch  0 Iteration  14 Loss:  1.8437827825546265
Epoch  0 Iteration  15 Loss:  1.8910883665084839
Epoch  0 Iteration  16 Loss:  1.7508856058120728
Epoch  0 Iteration  17 Loss:  1.9524320363998413
Epoch  0 Iteration  18 Loss:  1.851224660873413
Epoch  0 Iteration  19 Loss:  1.8271737098693848
Epoch  0 Iteration  20 Loss:  2.02211

In [0]:
import time

def get_classification(test_data_batches, trained_model, ce_loss):
  all_predictions = []
  all_true = []
  trained_model.eval()
  for epoch in range(1):
    epoch_loss = 0
    start = time.time()
    for i, sample in enumerate(test_data_batches):
      batchInput = sample['data']
      batchLabel = sample['label']
      batchLap = torch.Tensor(get_laplacian_for_batch(sample['data']))

      batchInput = batchInput.cuda()
      batchLabel = batchLabel.cuda()
      batchLap = batchLap.cuda()
      
      

      with torch.no_grad():
        batchOutput = trained_model(batchInput, batchLap)
      
      predictedLabel = torch.argmax(batchOutput)
      all_predictions.append(predictedLabel.item())
      all_true.append(batchLabel.item())
      print("Predicted Class : ", predictedLabel.item(), " True Label : ", batchLabel.item())

      loss = ce_loss(batchOutput, batchLabel.long())

      epoch_loss += loss.item()

      print("Epoch ", epoch, "Iteration ", i, "Loss: ", loss.item())
    end = time.time()
    time_per_epoch = end - start
    epoch_loss /= len(test_data_batches)
    all_predictions = np.array(all_predictions)
    all_true = np.array(all_true)
    accuracy = 1 - (np.count_nonzero(all_predictions - all_true) / len(all_predictions))
    
    # accuracy = (all_predictions == all_true).float().sum() / len(all_predictions)
    print("*********************************************************************")
    print("Test Loss ==>  ", epoch_loss, " Accuracy ==> ", accuracy,  " Time taken ==> ", time_per_epoch)
    print("*********************************************************************")
  return all_predictions          

In [0]:
ce_loss = nn.CrossEntropyLoss()
all_preds = get_classification(test_data_batches, gcn_model, ce_loss)

Predicted Class :  4  True Label :  4.0
Epoch  0 Iteration  0 Loss:  0.032591819763183594
Predicted Class :  0  True Label :  0.0
Epoch  0 Iteration  1 Loss:  0.005115032196044922
Predicted Class :  2  True Label :  2.0
Epoch  0 Iteration  2 Loss:  0.2283463478088379
Predicted Class :  8  True Label :  8.0
Epoch  0 Iteration  3 Loss:  0.014149665832519531
Predicted Class :  0  True Label :  0.0
Epoch  0 Iteration  4 Loss:  0.005074977874755859
Predicted Class :  8  True Label :  8.0
Epoch  0 Iteration  5 Loss:  0.00011920928955078125
Predicted Class :  2  True Label :  1.0
Epoch  0 Iteration  6 Loss:  2.3232994079589844
Predicted Class :  4  True Label :  4.0
Epoch  0 Iteration  7 Loss:  0.1092233657836914
Predicted Class :  4  True Label :  4.0
Epoch  0 Iteration  8 Loss:  0.0040721893310546875
Predicted Class :  0  True Label :  0.0
Epoch  0 Iteration  9 Loss:  0.006213665008544922
Predicted Class :  7  True Label :  7.0
Epoch  0 Iteration  10 Loss:  0.15484952926635742
Predicted Cla

In [0]:
torch.save(trained_gcn_model, '/content/trained_gcn.pth')
torch.save(trained_gcn_model.state_dict(), '/content/trained_gcn_state_dict.pth')


/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type pointGCN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type gcnLayer. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type ReLU. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Dropout. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist

In [0]:
gcn_model = pointGCN(para)
# gcn_model.load_state_dict(torch.load('/content/gdrive/My Drive/CIS_565_2019/trained_gcn_state_dict.pth'))
gcn_model = torch.load('/content/gdrive/My Drive/CIS_565_2019/trained_gcn.pth')
gcn_model = gcn_model.to(device)
gcn_model.eval()

pointGCN(
  (gcn1_layer): gcnLayer(
    (relu): ReLU()
  )
  (gcn1_dropout_layer): Dropout(p=0.09999999999999998, inplace=False)
  (gcn1_global_pooling_layer): globalPooling()
  (gcn2_layer): gcnLayer(
    (relu): ReLU()
  )
  (gcn2_dropout_layer): Dropout(p=0.09999999999999998, inplace=False)
  (gcn2_global_pooling_layer): globalPooling()
  (global_features_dropout_layer): Dropout(p=0.44999999999999996, inplace=False)
  (fc1): Linear(in_features=4000, out_features=600, bias=True)
  (relu): ReLU()
  (fc1_droput): Dropout(p=0.44999999999999996, inplace=False)
  (fc2): Linear(in_features=600, out_features=10, bias=True)
)

In [0]:
def point_wise_global_pooling(ip):
  var = torch.var(ip, dim=1) ** 2
  var = var.repeat(ip.shape[1], 1)
  op = torch.cat((var, ip.squeeze()), dim = 1)
  return op

def point_wise_forward(gcn_model, inputPC, scaledLaplacian):
  gcn_1         = gcn_model.gcn1_layer(inputPC, scaledLaplacian)
  gcn_1_output  = gcn_model.gcn1_dropout_layer(gcn_1)
  
  gcn_1_pooling = point_wise_global_pooling(gcn_1_output)
  
  gcn_2         = gcn_model.gcn2_layer(gcn_1_output, scaledLaplacian)
  gcn_2_output  = gcn_model.gcn2_dropout_layer(gcn_2)
  gcn_2_pooling = point_wise_global_pooling(gcn_2_output)

  globalFeatures = torch.cat((gcn_1_pooling, gcn_2_pooling), dim=1)
  globalFeatures = gcn_model.global_features_dropout_layer(globalFeatures)

  # fully connected layer 1
  fc_layer_1 = gcn_model.fc1(globalFeatures)
  fc_layer_1 = gcn_model.relu(fc_layer_1)
  fc_layer_1 = gcn_model.fc1_droput(fc_layer_1)

  # fully connected layer 2
  fc_layer_2 = gcn_model.fc2(fc_layer_1)

  return fc_layer_2

In [0]:
def point_wise_classification(inputTrainTensor, gcn_model, indices):
  all_class = []
  for idx in range(len(indices)):
    x = inputTrainTensor[indices[idx]].unsqueeze(0)
    L = torch.Tensor(get_laplacian_for_batch(x))
    x = x.cuda()
    L = L.cuda()
    y = point_wise_forward(gcn_model, x, L)
    y = y.unsqueeze(0)
    y = torch.argmax(y, dim=2)
    all_class.append(y)
  all_class = torch.cat(all_class, dim=0)
  return all_class

indices = [i for i in range(500)]
all_class = point_wise_classification(inputTrainTensor, gcn_model, indices)
print(all_class.shape)

torch.Size([1, 1024])
torch.Size([1, 1024])
torch.Size([1, 1024])
torch.Size([1, 1024])
torch.Size([1, 1024])
torch.Size([1, 1024])
torch.Size([1, 1024])
torch.Size([1, 1024])
torch.Size([1, 1024])
torch.Size([1, 1024])
torch.Size([1, 1024])
torch.Size([1, 1024])
torch.Size([1, 1024])
torch.Size([1, 1024])
torch.Size([1, 1024])
torch.Size([1, 1024])
torch.Size([1, 1024])
torch.Size([1, 1024])
torch.Size([1, 1024])
torch.Size([1, 1024])
torch.Size([1, 1024])
torch.Size([1, 1024])
torch.Size([1, 1024])
torch.Size([1, 1024])
torch.Size([1, 1024])
torch.Size([1, 1024])
torch.Size([1, 1024])
torch.Size([1, 1024])
torch.Size([1, 1024])
torch.Size([1, 1024])
torch.Size([1, 1024])
torch.Size([1, 1024])
torch.Size([1, 1024])
torch.Size([1, 1024])
torch.Size([1, 1024])
torch.Size([1, 1024])
torch.Size([1, 1024])
torch.Size([1, 1024])
torch.Size([1, 1024])
torch.Size([1, 1024])
torch.Size([1, 1024])
torch.Size([1, 1024])
torch.Size([1, 1024])
torch.Size([1, 1024])
torch.Size([1, 1024])
torch.Size

In [0]:
np.save("./all_points.npy", inputTrainTensor.numpy())
np.save("./all_predictions.npy", all_class.cpu().numpy())

In [0]:
x = torch.rand(1, 1024, 1000)
y = point_wise_global_pooling(x)
print(y.shape)

torch.Size([1024, 2000])


In [0]:
def get_point_wise_classification(sample, sampleLabel, trained_model, ce_loss):
  all_predictions = []
  all_true = []
  trained_model.eval()
  
  for epoch in range(1):
    epoch_loss = 0
    start = time.time()
    for i, sample in enumerate(test_data_batches):
      batchInput = sample['data']
      batchLabel = sample['label']
      batchLap = torch.Tensor(get_laplacian_for_batch(sample['data']))

      batchInput = batchInput.cuda()
      batchLabel = batchLabel.cuda()
      batchLap = batchLap.cuda()
      
      

      with torch.no_grad():
        batchOutput = trained_model(batchInput, batchLap)
      
      predictedLabel = torch.argmax(batchOutput)
      all_predictions.append(predictedLabel.item())
      all_true.append(batchLabel.item())
      print("Predicted Class : ", predictedLabel.item(), " True Label : ", batchLabel.item())

      loss = ce_loss(batchOutput, batchLabel.long())

      epoch_loss += loss.item()

      print("Epoch ", epoch, "Iteration ", i, "Loss: ", loss.item())
    end = time.time()
    time_per_epoch = end - start
    epoch_loss /= len(test_data_batches)
    all_predictions = np.array(all_predictions)
    all_true = np.array(all_true)
    accuracy = 1 - (np.count_nonzero(all_predictions - all_true) / len(all_predictions))
    
    # accuracy = (all_predictions == all_true).float().sum() / len(all_predictions)
    print("*********************************************************************")
    print("Test Loss ==>  ", epoch_loss, " Accuracy ==> ", accuracy,  " Time taken ==> ", time_per_epoch)
    print("*********************************************************************")
  return all_predictions 